In [1]:
!pip3 install gensim

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import sys
sys.path.insert(0, '../preprocessing')

In [3]:
from loader import read_replacements
from transformers import StopWordRemover, SemanticMapper, LowerCaseSentence
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline

[nltk_data] Downloading package punkt to /Users/maxyuan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maxyuan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_csv('../data/inputs/ct_desc.csv')
df.head()

,PID,Exam Description,Hashtag,Impression Cleaned
0,1,CT ANGIOGRAM CHEST,"[unchanged]_intramural_hematoma, ascending_aor...",IMPRESSION:\n1. Unchanged descending thoracic...
1,2,CT CHEST WITH CONTRAST,"[unchange]_[right]_pleural effusion, [decrease...",IMPRESSION:\n1. Slight decrease in the left lu...
2,3,CT CHEST PULMONARY EMBOLISM (CTPE),"[negx]_pulmonary_embolism, [decrease]_[bilater...",IMPRESSION:\n1. No pulmonary embolism.\n2. Sli...
3,4,CT CHEST WITH CONTRAST,[negex]_metastasis,IMPRESSION:\nNo intrathoracic metastatic disea...
4,5,CT CHEST WITH CONTRAST,"[left]_[supraclavicular]_LN, [risk]_metastasis",IMPRESSION:\nLarge left heterogeneously enhanc...


In [5]:
df['Impression Cleaned'][0]

'IMPRESSION:\n1.  Unchanged descending thoracic aorta intramural hematoma with persistent intramural blood pool measuring approximately 1.4 cm in size, which is supplied by an intercostal artery.\n2.  Unchanged enlargement of the ascending aorta measuring up to 4.1 cm, with diffuse calcified and noncalcified atherosclerotic plaque throughout the thoracic aorta.\n3.  Unchanged 2.3 cm left thyroid nodule which warrants dedicated thyroid ultrasound if not previously performed.\n'

In [6]:
"""
with open('impressions.txt', 'w+') as f:
    for row in df['Impression Cleaned']:
        f.write(row)
        
with open('mini_impression.txt', 'w+') as f:
    for row in df['Impression Cleaned'][10]:
        f.write(row)
"""

"\nwith open('impressions.txt', 'w+') as f:\n    for row in df['Impression Cleaned']:\n        f.write(row)\n        \nwith open('mini_impression.txt', 'w+') as f:\n    for row in df['Impression Cleaned'][10]:\n        f.write(row)\n"

In [7]:
content = []
with open('mini_impression.txt', 'r') as f:
    for line in f:
        content.append(line)

In [8]:
content

['IMPRESSION:\n',
 '1.  Unchanged descending thoracic aorta intramural hematoma with persistent intramural blood pool measuring approximately 1.4 cm in size, which is supplied by an intercostal artery.\n',
 '2.  Unchanged enlargement of the ascending aorta measuring up to 4.1 cm, with diffuse calcified and noncalcified atherosclerotic plaque throughout the thoracic aorta.\n',
 '3.  Unchanged 2.3 cm left thyroid nodule which warrants dedicated thyroid ultrasound if not previously performed.\n',
 'IMPRESSION:\n',
 '1. Slight decrease in the left lung and pleural metastatic disease.\n',
 '2.  Unchanged small right pleural effusion.\n',
 'IMPRESSION:\n',
 '1. No pulmonary embolism.\n',
 '2. Slight interval decrease in scattered groundglass opacities and interlobular septal thickening, compatible with pulmonary edema which is slightly improving.\n',
 '3. Medium right and small left pleural effusion. Small pericardial effusion. These findings are similar to prior and are suggestive of volume

In [9]:
print(word_tokenize(content[14]))

['Large', 'left', 'heterogeneously', 'enhancing', 'supraclavicular', 'lymph', 'node', 'likely', 'represents', 'a', 'metastasis', 'via', 'the', 'thoracic', 'duct', 'drainage', 'in', 'the', 'setting', 'of', 'retroperitoneal', 'lymphadenopathy', '(', 'Virchow', 'node', ')', '.', 'No', 'additional', 'findings', 'indicate', 'intrathoracic', 'metastasis', '.']


In [10]:
DateTimeMapper = SemanticMapper([(r'[0-9][0-9]? [0-9][0-9]? [0-9][0-9][0-9][0-9]', ''),
                                 (r'[0-9][0-9]? [0-9][0-9] (am|pm)?', '')], regex=True)
AlphaNumRemover = SemanticMapper([(r' [0-9]+', '')], regex=True)
ExtenderPreserver = SemanticMapper([(' or ', ' EXT '), (' nor ', ' EXT ')])
ExtenderRemover = SemanticMapper(
    [('EXT', ''), ('NEGEX_EXT', ''), (('NEGEX_ ', ''))])

In [11]:
content[1]

'1.  Unchanged descending thoracic aorta intramural hematoma with persistent intramural blood pool measuring approximately 1.4 cm in size, which is supplied by an intercostal artery.\n'

In [12]:
print([content[1]])
swr = StopWordRemover()
labeled_output = swr.transform([[content[1]]])
print(labeled_output)

['1.  Unchanged descending thoracic aorta intramural hematoma with persistent intramural blood pool measuring approximately 1.4 cm in size, which is supplied by an intercostal artery.\n']
words ['1', '.', 'Unchanged', 'descending', 'thoracic', 'aorta', 'intramural', 'hematoma', 'with', 'persistent', 'intramural', 'blood', 'pool', 'measuring', 'approximately', '1.4', 'cm', 'in', 'size', ',', 'which', 'is', 'supplied', 'by', 'an', 'intercostal', 'artery', '.']
filtered ['1', '.', 'Unchanged', 'descending', 'thoracic', 'aorta', 'intramural', 'hematoma', 'persistent', 'intramural', 'blood', 'pool', 'measuring', 'approximately', '1.4', 'cm', 'size', ',', 'supplied', 'intercostal', 'artery', '.']
[['1 . Unchanged descending thoracic aorta intramural hematoma persistent intramural blood pool measuring approximately 1.4 cm size , supplied intercostal artery .']]


Shape of the input list of sentences in list of reports

In [13]:
pipeline = make_pipeline(StopWordRemover(), None)
labeled_output = pipeline.transform([content])
labeled_output

words ['IMPRESSION', ':']
filtered ['IMPRESSION', ':']
words ['1', '.', 'Unchanged', 'descending', 'thoracic', 'aorta', 'intramural', 'hematoma', 'with', 'persistent', 'intramural', 'blood', 'pool', 'measuring', 'approximately', '1.4', 'cm', 'in', 'size', ',', 'which', 'is', 'supplied', 'by', 'an', 'intercostal', 'artery', '.']
filtered ['1', '.', 'Unchanged', 'descending', 'thoracic', 'aorta', 'intramural', 'hematoma', 'persistent', 'intramural', 'blood', 'pool', 'measuring', 'approximately', '1.4', 'cm', 'size', ',', 'supplied', 'intercostal', 'artery', '.']
words ['2', '.', 'Unchanged', 'enlargement', 'of', 'the', 'ascending', 'aorta', 'measuring', 'up', 'to', '4.1', 'cm', ',', 'with', 'diffuse', 'calcified', 'and', 'noncalcified', 'atherosclerotic', 'plaque', 'throughout', 'the', 'thoracic', 'aorta', '.']
filtered ['2', '.', 'Unchanged', 'enlargement', 'ascending', 'aorta', 'measuring', '4.1', 'cm', ',', 'diffuse', 'calcified', 'noncalcified', 'atherosclerotic', 'plaque', 'througho

[['IMPRESSION :',
  '1 . Unchanged descending thoracic aorta intramural hematoma persistent intramural blood pool measuring approximately 1.4 cm size , supplied intercostal artery .',
  '2 . Unchanged enlargement ascending aorta measuring 4.1 cm , diffuse calcified noncalcified atherosclerotic plaque throughout thoracic aorta .',
  '3 . Unchanged 2.3 cm left thyroid nodule warrants dedicated thyroid ultrasound previously performed .',
  'IMPRESSION :',
  '1 . Slight decrease left lung pleural metastatic disease .',
  '2 . Unchanged small right pleural effusion .',
  'IMPRESSION :',
  '1 . No pulmonary embolism .',
  '2 . Slight interval decrease scattered groundglass opacities interlobular septal thickening , compatible pulmonary edema slightly improving .',
  '3 . Medium right small left pleural effusion . Small pericardial effusion . These findings similar prior suggestive volume overload .',
  'IMPRESSION :',
  'No intrathoracic metastatic disease .',
  'IMPRESSION :',
  'Large left

In [14]:
rep = read_replacements('../data/semantic_maps/radlex_replacements')
lower = LowerCaseSentence()
radlex_replacements = SemanticMapper(rep)

In [15]:
pipeline = make_pipeline(lower, None)
labeled_output = pipeline.transform([content])
labeled_output

[['impression:\n',
  '1.  unchanged descending thoracic aorta intramural hematoma with persistent intramural blood pool measuring approximately 1.4 cm in size, which is supplied by an intercostal artery.\n',
  '2.  unchanged enlargement of the ascending aorta measuring up to 4.1 cm, with diffuse calcified and noncalcified atherosclerotic plaque throughout the thoracic aorta.\n',
  '3.  unchanged 2.3 cm left thyroid nodule which warrants dedicated thyroid ultrasound if not previously performed.\n',
  'impression:\n',
  '1. slight decrease in the left lung and pleural metastatic disease.\n',
  '2.  unchanged small right pleural effusion.\n',
  'impression:\n',
  '1. no pulmonary embolism.\n',
  '2. slight interval decrease in scattered groundglass opacities and interlobular septal thickening, compatible with pulmonary edema which is slightly improving.\n',
  '3. medium right and small left pleural effusion. small pericardial effusion. these findings are similar to prior and are suggestiv

In [16]:
pipeline = make_pipeline(radlex_replacements, None)
labeled_output = pipeline.transform([content])
labeled_output

[['impression:\n',
  '1. unchanged descending_thoracic_aorta intramural hematoma with persistent intramural blood pool measuring approximately 1.4 curium in size, which is supplied by an intercostal artery.\n',
  '2. unchanged enlargement of the ascending_aorta measuring up to 4.1 cm, with diffuse calcified and noncalcified atherosclerotic_plaque throughout the thoracic aorta.\n',
  '3. unchanged 2.3 curium left thyroid_gland breast_mass which warrants dedicated thyroid_gland ultrasound if not previously performed.\n',
  'impression:\n',
  '1. slight decrease in the left_lung and pleural metastatic disease.\n',
  '2. unchanged small right pleural effusion.\n',
  'impression:\n',
  '1. no pulmonary embolism.\n',
  '2. slight interval decrease in scattered groundglass opacities and interlobular septal thickening, compatible with pulmonary_edema which is slightly improving.\n',
  '3. medium right and small left pleural effusion. small pericardial effusion. these findings are similar to pr

In [17]:
clever_r = read_replacements('../data/semantic_maps/clever_replacements')
misc_r = read_replacements('../data/semantic_maps/misc_replacements')


In [18]:
clever_r

[(' . ', ' DOT '),
 (' , ', ' PUNCT '),
 (' ; ', ' PUNCT '),
 (' : ', ' PUNCT '),
 (' ! ', ' PUNCT '),
 (' ? ', ' PUNCT '),
 (' patient ', ' PT '),
 (' pt ', ' PT '),
 (" pt's ", ' PT '),
 (' care giver ', ' CG '),
 (' caregiver ', ' CG '),
 (' aunt ', ' FAM '),
 (" aunt's ", ' FAM '),
 (' aunts ', ' FAM '),
 (" aunts' ", ' FAM '),
 (' boyfriend ', ' FAM '),
 (" boyfriend's ", ' FAM '),
 (' boyfriends ', ' FAM '),
 (" boyfriends' ", ' FAM '),
 (' bride ', ' FAM '),
 (" bride's ", ' FAM '),
 (' bridegroom ', ' FAM '),
 (" bridegroom's ", ' FAM '),
 (' bridegrooms ', ' FAM '),
 (" bridegrooms' ", ' FAM '),
 (' brides ', ' FAM '),
 (" brides' ", ' FAM '),
 (' brother ', ' FAM '),
 (' brother-in-law ', ' FAM '),
 (" brother-in-law's ", ' FAM '),
 (" brother-in-laws' ", ' FAM '),
 (" brother's ", ' FAM '),
 (' brothers ', ' FAM '),
 (' brothers-in-law ', ' FAM '),
 (" brothers' ", ' FAM '),
 (' child ', ' FAM '),
 (" child's ", ' FAM '),
 (' children ', ' FAM '),
 (' childs ', ' FAM '),
 ("

In [19]:
pipeline = make_pipeline(SemanticMapper(clever_r), None)
labeled_output = pipeline.transform([content])
labeled_output

[['impression:\n',
  '1. unchanged descending thoracic aorta intramural hematoma with persistent intramural blood pool measuring approximately 1.4 cm in size, which is supplied by an intercostal artery.\n',
  '2. unchanged enlargement of the ascending aorta measuring up to 4.1 cm, with diffuse calcified and noncalcified atherosclerotic plaque throughout the thoracic aorta.\n',
  '3. unchanged 2.3 cm left thyroid nodule which warrants dedicated thyroid ultrasound if NEGEX previously performed.\n',
  'impression:\n',
  '1. slight decrease in the left lung and pleural metastatic disease.\n',
  '2. unchanged small right pleural effusion.\n',
  'impression:\n',
  '1. NEGEX pulmonary embolism.\n',
  '2. slight interval decrease in scattered groundglass opacities and interlobular septal thickening, compatible with pulmonary edema which is slightly improving.\n',
  '3. medium right and small left pleural effusion. small pericardial effusion. these findings are similar to prior and are RISK of 